In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-94b50cf1-dccd-03a1-ee83-9e458e7f4cfe)


Import các thư viện cần sử dụng

In [2]:
!pip install pyvi
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.

In [16]:
import json
import math
from typing import List, Dict, Tuple, Optional
import re
from collections import Counter, defaultdict
import os
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from itertools import combinations
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer
from pyvi import ViTokenizer
from tqdm import tqdm

Định nghĩa class RankNet - kết thừa từ thư viện: torch.nn.Module

In [39]:

class RankNet(nn.Module):
    def __init__(self, input_size, hidden_size1=256, hidden_size2=128, dropout=0.2):
        """
        RankNet với 2 tầng ẩn

        Args:
            input_size (int): Số features đầu vào
            hidden_size1 (int): Số neurons tầng ẩn thứ nhất
            hidden_size2 (int): Số neurons tầng ẩn thứ hai
            dropout (float): Tỷ lệ dropout để tránh overfitting
        """
        super(RankNet, self).__init__()

        # Định nghĩa các tầng
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, 1)  # Output layer cho score

        # Dropout layers
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

        # Layer normalization thay vì batch normalization để tránh lỗi khi batch_size=1
        self.ln1 = nn.LayerNorm(hidden_size1)
        self.ln2 = nn.LayerNorm(hidden_size2)

    def forward(self, x):
        """
        Forward pass

        Args:
            x (torch.Tensor): Input tensor có shape (batch_size, input_size)

        Returns:
            torch.Tensor: Output scores có shape (batch_size, 1)
        """
        # Đảm bảo input là tensor và có đúng kiểu dữ liệu
        if not isinstance(x, torch.Tensor):
            x = torch.tensor(x, dtype=torch.float32)

        # Tầng ẩn thứ nhất
        x = self.fc1(x)
        x = self.ln1(x)
        x = F.relu(x)
        x = self.dropout1(x)

        # Tầng ẩn thứ hai
        x = self.fc2(x)
        x = self.ln2(x)
        x = F.relu(x)
        x = self.dropout2(x)

        # Output layer
        x = self.fc3(x)

        return x

    def predict_rank(self, x1, x2):
        """
        So sánh ranking giữa hai samples

        Args:
            x1, x2 (torch.Tensor): Hai samples cần so sánh

        Returns:
            torch.Tensor: Xác suất x1 được rank cao hơn x2
        """
        score1 = self.forward(x1)
        score2 = self.forward(x2)

        # Sử dụng sigmoid để chuyển về xác suất
        prob = torch.sigmoid(score1 - score2)
        return prob


Hàm mất mát

In [32]:
def ranknet_loss(s_i, s_j, P_ij):
    diff = s_i - s_j
    P_hat = torch.sigmoid(diff)  # Xác suất dự đoán P̂ᵢⱼ
    # Thêm epsilon để tránh log(0)
    epsilon = 1e-10
    loss = -P_ij * torch.log(P_hat + epsilon) - (1 - P_ij) * torch.log(1 - P_hat + epsilon)
    return loss.mean()

Một số lớp, hàm phụ trợ

In [33]:
class DataPreparator:
    """
    Class chuyên dụng cho việc chuẩn bị dữ liệu pairwise training
    """

    def __init__(self, sentence_model: SentenceTransformer, stopwords_path: Optional[str] = None):
        """
        Khởi tạo DataPreparator

        Args:
            sentence_model: Mô hình sentence transformer
            stopwords_path: Đường dẫn file stopwords (optional)
        """
        self.sentence_model = sentence_model
        self.stopwords = set()

        # Load stopwords nếu có
        if stopwords_path and os.path.exists(stopwords_path):
            with open(stopwords_path, 'r', encoding='utf-8') as f:
                self.stopwords = set(line.strip() for line in f if line.strip())
            print(f"Đã load {len(self.stopwords)} stopwords từ {stopwords_path}")

    def preprocess_text(self, text: str) -> str:
        """
        Tiền xử lý văn bản với PyVi

        Args:
            text: Văn bản cần xử lý

        Returns:
            str: Văn bản đã được xử lý
        """
        # 1. Chuẩn hóa cơ bản
        text = text.lower()
        text = re.sub(r'[^\w\s]', ' ', text)  # loại bỏ dấu câu
        text = re.sub(r'\d+', ' ', text)      # loại bỏ số
        text = re.sub(r'\s+', ' ', text).strip()  # bỏ khoảng trắng thừa

        # 2. Tokenize với PyVi
        tokenized = ViTokenizer.tokenize(text)

        # 3. Tách thành danh sách từ và lọc stopwords
        tokens = tokenized.split()
        if self.stopwords:
            tokens = [token for token in tokens if token not in self.stopwords]

        return " ".join(tokens)

    def calculate_relevance_scores(self, query: str, documents: List[Dict],
                                 relevance_method: str = 'cosine_ranking') -> List[Tuple[int, float, int]]:
        """
        Tính điểm relevance cho tất cả documents với một query

        Args:
            query: Câu truy vấn
            documents: Danh sách documents
            relevance_method: Phương pháp tính relevance

        Returns:
            List[Tuple[int, float, int]]: (doc_index, similarity_score, relevance_label)
        """
        # Preprocess query
        processed_query = self.preprocess_text(query)

        # Encode query
        query_embedding = self.sentence_model.encode([processed_query])

        # Preprocess và encode tất cả documents
        doc_texts = []
        for doc in documents:
            processed_doc = self.preprocess_text(doc['value'])
            doc_texts.append(processed_doc)

        doc_embeddings = self.sentence_model.encode(doc_texts)

        # Tính cosine similarity
        similarities = cosine_similarity(query_embedding, doc_embeddings)[0]

        # Gán relevance labels
        doc_relevances = []

        if relevance_method == 'cosine_ranking':
            # Sắp xếp theo similarity và gán labels theo ranking
            doc_similarities = [(i, sim) for i, sim in enumerate(similarities)]
            doc_similarities.sort(key=lambda x: x[1], reverse=True)

            for rank, (doc_idx, similarity) in enumerate(doc_similarities):
                if rank < 3:  # Top 1-3: Highly relevant
                    relevance = 3
                elif rank < 7:  # Top 4-7: Relevant
                    relevance = 2
                elif rank < 12:  # Top 8-12: Somewhat relevant
                    relevance = 1
                else:  # Còn lại: Not relevant
                    relevance = 0

                doc_relevances.append((doc_idx, float(similarity), relevance))

        elif relevance_method == 'threshold_based':
            # Gán labels dựa trên threshold của similarity
            for doc_idx, similarity in enumerate(similarities):
                if similarity >= 0.8:
                    relevance = 3
                elif similarity >= 0.6:
                    relevance = 2
                elif similarity >= 0.4:
                    relevance = 1
                else:
                    relevance = 0

                doc_relevances.append((doc_idx, float(similarity), relevance))

        return doc_relevances

    def generate_pairwise_data(self, queries: List[str], documents: List[Dict],
                             relevance_method: str = 'cosine_ranking',
                             max_pairs_per_query: Optional[int] = None,
                             balance_labels: bool = True) -> List[Dict]:
        """
        Tạo dữ liệu pairwise training

        Args:
            queries: Danh sách queries
            documents: Danh sách documents
            relevance_method: Phương pháp tính relevance
            max_pairs_per_query: Giới hạn số cặp per query (optional)
            balance_labels: Có cân bằng labels không

        Returns:
            List[Dict]: Danh sách pairwise samples
        """
        print(f"Bắt đầu tạo dữ liệu pairwise với {len(queries)} queries và {len(documents)} documents")

        all_pairwise_data = []
        label_counts = {0.0: 0, 0.5: 0, 1.0: 0}

        for query_idx, query in enumerate(tqdm(queries, desc="Tạo dữ liệu pairwise")):
            # Tính relevance scores
            doc_relevances = self.calculate_relevance_scores(query, documents, relevance_method)

            # Tạo tất cả các cặp possible
            query_pairs = []

            for i in range(len(doc_relevances)):
                for j in range(i + 1, len(doc_relevances)):
                    doc1_idx, doc1_sim, doc1_rel = doc_relevances[i]
                    doc2_idx, doc2_sim, doc2_rel = doc_relevances[j]

                    # Xác định label cho cặp
                    if doc1_rel > doc2_rel:
                        label = 1.0  # Doc1 tốt hơn Doc2
                    elif doc1_rel < doc2_rel:
                        label = 0.0  # Doc1 kém hơn Doc2
                    else:
                        label = 0.5  # Bằng nhau

                    # Tạo sample
                    pairwise_sample = {
                        "query_id": query_idx,
                        "query": query,
                        "query_processed": self.preprocess_text(query),
                        "doc1_id": documents[doc1_idx]['id'],
                        "doc1_text": documents[doc1_idx]['value'],
                        "doc1_processed": self.preprocess_text(documents[doc1_idx]['value']),
                        "doc1_relevance": doc1_rel,
                        "doc1_similarity": doc1_sim,
                        "doc2_id": documents[doc2_idx]['id'],
                        "doc2_text": documents[doc2_idx]['value'],
                        "doc2_processed": self.preprocess_text(documents[doc2_idx]['value']),
                        "doc2_relevance": doc2_rel,
                        "doc2_similarity": doc2_sim,
                        "label": label,
                        "relevance_method": relevance_method
                    }

                    query_pairs.append(pairwise_sample)
                    label_counts[label] += 1

            # Giới hạn số cặp per query nếu cần
            if max_pairs_per_query and len(query_pairs) > max_pairs_per_query:
                # Sampling có cân bằng labels
                if balance_labels:
                    query_pairs = self._balanced_sampling(query_pairs, max_pairs_per_query)
                else:
                    query_pairs = query_pairs[:max_pairs_per_query]

            all_pairwise_data.extend(query_pairs)

        # In thống kê
        total_pairs = len(all_pairwise_data)
        print(f"\nThống kê dữ liệu pairwise:")
        print(f"Tổng số cặp: {total_pairs}")
        print(f"Label 0.0 (doc1 < doc2): {label_counts[0.0]} ({label_counts[0.0]/total_pairs*100:.1f}%)")
        print(f"Label 0.5 (doc1 = doc2): {label_counts[0.5]} ({label_counts[0.5]/total_pairs*100:.1f}%)")
        print(f"Label 1.0 (doc1 > doc2): {label_counts[1.0]} ({label_counts[1.0]/total_pairs*100:.1f}%)")

        return all_pairwise_data

    def _balanced_sampling(self, pairs: List[Dict], max_pairs: int) -> List[Dict]:
        """
        Lấy mẫu cân bằng theo labels

        Args:
            pairs: Danh sách tất cả các cặp
            max_pairs: Số cặp tối đa

        Returns:
            List[Dict]: Danh sách cặp đã được sampling
        """
        # Phân loại theo labels
        label_groups = {0.0: [], 0.5: [], 1.0: []}
        for pair in pairs:
            label_groups[pair['label']].append(pair)

        # Tính số mẫu cho mỗi label (cân bằng)
        samples_per_label = max_pairs // 3
        remaining = max_pairs % 3

        sampled_pairs = []

        # Lấy mẫu từng label
        for i, (label, group) in enumerate(label_groups.items()):
            n_samples = samples_per_label + (1 if i < remaining else 0)
            n_samples = min(n_samples, len(group))  # Không vượt quá số có sẵn

            # Random sampling
            import random
            sampled = random.sample(group, n_samples)
            sampled_pairs.extend(sampled)

        return sampled_pairs

    def save_pairwise_data(self, pairwise_data: List[Dict], file_path: str,
                          metadata: Optional[Dict] = None):
        """
        Lưu dữ liệu pairwise vào file

        Args:
            pairwise_data: Dữ liệu pairwise
            file_path: Đường dẫn file
            metadata: Metadata bổ sung (optional)
        """
        # Tạo thư mục nếu chưa có
        os.makedirs(os.path.dirname(file_path) if os.path.dirname(file_path) else '.', exist_ok=True)

        # Chuẩn bị data để lưu
        data_to_save = {
            "metadata": {
                "total_pairs": len(pairwise_data),
                "created_by": "DataPreparator",
                "format_version": "1.0",
                **(metadata or {})
            },
            "pairwise_data": pairwise_data
        }

        # Lưu file
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data_to_save, f, ensure_ascii=False, indent=2)

        print(f"Đã lưu {len(pairwise_data)} cặp dữ liệu vào {file_path}")

    def load_pairwise_data(self, file_path: str) -> Tuple[List[Dict], Dict]:
        """
        Load dữ liệu pairwise từ file

        Args:
            file_path: Đường dẫn file

        Returns:
            Tuple[List[Dict], Dict]: (pairwise_data, metadata)
        """
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File không tồn tại: {file_path}")

        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        pairwise_data = data.get('pairwise_data', [])
        metadata = data.get('metadata', {})

        print(f"Đã load {len(pairwise_data)} cặp dữ liệu từ {file_path}")

        return pairwise_data, metadata
def prepare_training_data(documents_path: str, queries: List[str],
                         output_file: str = "document_pairs.json",
                         sentence_model_name: str = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
                         stopwords_path: Optional[str] = None,
                         relevance_method: str = 'cosine_ranking',
                         max_pairs_per_query: Optional[int] = None):
    """
    Hàm wrapper để chuẩn bị dữ liệu training một cách dễ dàng

    Args:
        documents_path: Đường dẫn file documents JSON
        queries: Danh sách queries
        output_file: Tên file output
        sentence_model_name: Tên sentence transformer model
        stopwords_path: Đường dẫn stopwords file
        relevance_method: Phương pháp tính relevance
        max_pairs_per_query: Giới hạn cặp per query

    Returns:
        List[Dict]: Dữ liệu pairwise đã tạo
    """
    print("="*60)
    print("CHUẨN BỊ DỮ LIỆU PAIRWISE CHO RANKNET")
    print("="*60)

    # 1. Load documents
    print(f"1. Đang load documents từ {documents_path}...")
    with open(documents_path, 'r', encoding='utf-8') as f:
        documents = json.load(f)
    print(f"   ✓ Đã load {len(documents)} documents")

    # 2. Khởi tạo sentence transformer
    print(f"2. Đang khởi tạo sentence transformer: {sentence_model_name}...")
    sentence_model = SentenceTransformer(sentence_model_name)
    print(f"   ✓ Model đã sẵn sàng")

    # 3. Khởi tạo DataPreparator
    print("3. Đang khởi tạo DataPreparator...")
    preparator = DataPreparator(sentence_model, stopwords_path)
    print(f"   ✓ DataPreparator đã sẵn sàng")

    # 4. Tạo pairwise data
    print("4. Đang tạo dữ liệu pairwise...")
    pairwise_data = preparator.generate_pairwise_data(
        queries=queries,
        documents=documents,
        relevance_method=relevance_method,
        max_pairs_per_query=max_pairs_per_query,
        balance_labels=True
    )
    print(f"   ✓ Đã tạo {len(pairwise_data)} cặp dữ liệu")

    # 5. Lưu file
    print(f"5. Đang lưu vào {output_file}...")
    metadata = {
        "documents_path": documents_path,
        "num_queries": len(queries),
        "queries": queries,
        "relevance_method": relevance_method,
        "sentence_model": sentence_model_name,
        "max_pairs_per_query": max_pairs_per_query
    }

    preparator.save_pairwise_data(pairwise_data, output_file, metadata)
    print(f"   ✓ Dữ liệu đã được lưu thành công")

    print("="*60)
    print("HOÀN THÀNH CHUẨN BỊ DỮ LIỆU!")
    print("="*60)

    return pairwise_data

In [20]:
# Chuẩn bị dữ liệu, lấy ra so sánh
class RankNetDataset(Dataset):
    """Dataset cho RankNet training"""
    def __init__(self, pairwise_data, sentence_model):
        self.data = pairwise_data
        self.sentence_model = sentence_model

    def __len__(self):
        return len(self.data)

    #  Trả về một dictionary chứa feature1, feature2, và label
    def __getitem__(self, idx):
        sample = self.data[idx]

        # Encode query và documents
        query_emb = self.sentence_model.encode([sample['query']])[0]
        doc1_emb = self.sentence_model.encode([sample['doc1_text']])[0]
        doc2_emb = self.sentence_model.encode([sample['doc2_text']])[0]

        # Tạo features bằng cách concat query+doc
        feature1 = np.concatenate([query_emb, doc1_emb])
        feature2 = np.concatenate([query_emb, doc2_emb])

        return {
            'feature1': torch.tensor(feature1, dtype=torch.float32),
            'feature2': torch.tensor(feature2, dtype=torch.float32),
            'label': torch.tensor(sample['label'], dtype=torch.float32)
        }


def evaluate_ranking(model, documents, queries, sentence_model, preparator, device, top_k=10):
    """
    Đánh giá mô hình bằng cách rank lại documents cho mỗi query
    và tính pairwise accuracy
    """
    model.eval()
    model.to(device)

    results = {}

    with torch.no_grad():
        for query in queries:
            # Tiền xử lý query
            processed_query_tokens = preparator.preprocess_text(query)
            processed_query = " ".join(processed_query_tokens)

            # Encode query
            query_emb = sentence_model.encode([processed_query])[0]

            # Score tất cả documents
            doc_scores = []
            for doc in documents:
                processed_doc_tokens = preparator.preprocess_text(doc['value'])
                processed_doc = " ".join(processed_doc_tokens)

                doc_emb = sentence_model.encode([processed_doc])[0]

                feature = torch.tensor(
                    np.concatenate([query_emb, doc_emb]),
                    dtype=torch.float32
                ).unsqueeze(0).to(device)

                score = model(feature).item()
                doc_scores.append((doc['id'], score, doc))

            # Sắp xếp giảm dần theo score
            doc_scores.sort(key=lambda x: x[1], reverse=True)

            # Lấy top k
            top_docs = doc_scores[:top_k]

            results[query] = {
                'top_documents': [(doc_id, doc['value'], score) for doc_id, score, doc in top_docs],
                'all_scores': doc_scores
            }

    return results





In [30]:

# Hàm huấn luyện
def train_ranknet(model, dataloader, optimizer, device, num_epochs=20,
                          loss_type='cross_entropy', patience=5, min_delta=1e-4):
    """
    Huấn luyện mô hình RankNet với các cải tiến

    Args:
        model: RankNet model
        dataloader: DataLoader
        optimizer: optimizer
        device: cuda/cpu
        num_epochs: số epochs
        loss_type: loại loss function
        patience: số epochs chờ để early stopping
        min_delta: threshold nhỏ nhất để coi là improvement
    """
    model.train()
    model.to(device)

    # Learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=3, verbose=True, min_lr=1e-6
    )

    # Early stopping variables
    best_loss = float('inf')
    patience_counter = 0
    loss_history = []

    for epoch in range(num_epochs):
        total_loss = 0
        num_batches = 0

        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")

        for batch in progress_bar:
            feature1 = batch['feature1'].to(device)
            feature2 = batch['feature2'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()

            # Forward pass
            score1 = model(feature1).squeeze()
            score2 = model(feature2).squeeze()

            # Tính loss với hàm cải tiến
            loss = ranknet_loss(score1, score2, labels)

            # Backward pass
            loss.backward()

            # Gradient clipping để tránh exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            total_loss += loss.item()
            num_batches += 1

            progress_bar.set_postfix({
                'Loss': f"{loss.item():.4f}",
                'LR': f"{optimizer.param_groups[0]['lr']:.6f}"
            })

        avg_loss = total_loss / num_batches
        loss_history.append(avg_loss)

        # Learning rate scheduling
        scheduler.step(avg_loss)

        print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")

        # Early stopping check
        if avg_loss < best_loss - min_delta:
            best_loss = avg_loss
            patience_counter = 0
            # Save best model
            torch.save(model.state_dict(), 'best_ranknet_model.pth')
            print(f"New best model saved with loss: {best_loss:.4f}")
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs")
            print(f"Loading best model with loss: {best_loss:.4f}")
            model.load_state_dict(torch.load('best_ranknet_model.pth'))
            break

    return model

In [31]:
# Hàm đánh giá kết quả
def calculate_pairwise_accuracy(model, test_data, sentence_model, device):
    """Tính Pairwise Accuracy"""
    model.eval()
    model.to(device)

    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for sample in tqdm(test_data, desc="Tính Pairwise Accuracy"):
            # Encode features
            query_emb = sentence_model.encode([sample['query']])[0]
            doc1_emb = sentence_model.encode([sample['doc1_text']])[0]
            doc2_emb = sentence_model.encode([sample['doc2_text']])[0]

            feature1 = torch.tensor(np.concatenate([query_emb, doc1_emb]),
                                  dtype=torch.float32).unsqueeze(0).to(device)
            feature2 = torch.tensor(np.concatenate([query_emb, doc2_emb]),
                                  dtype=torch.float32).unsqueeze(0).to(device)

            # Tính scores
            score1 = model(feature1).item()
            score2 = model(feature2).item()

            # Dự đoán
            if sample['label'] == 1.0:  # doc1 should be ranked higher
                if score1 > score2:
                    correct_predictions += 1
            elif sample['label'] == 0.0:  # doc2 should be ranked higher
                if score2 > score1:
                    correct_predictions += 1
            else:  # Equal relevance (label = 0.5)
                # Coi như đúng nếu difference nhỏ
                if abs(score1 - score2) < 0.1:
                    correct_predictions += 1

            total_predictions += 1

    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
    return accuracy

**Chạy chương trình** Tính toán chính

In [36]:
if __name__ == "__main__":
    try:
        # Thiết lập device
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Sử dụng device: {device}")

        # Bước 1: Tải tài liệu và khởi tạo sentence-transformer
        documents_path = "/content/cadao_tucngu_50_1.json"
        pairwise_data_file = "/content/document_pairs.json"
        # documents_path = "FetchData/train/anh_em_mot_nha/data.json"

        # Chạy document để lấy đánh giá với kết quả
        with open(documents_path, 'r', encoding='utf-8') as f:
            documents = json.load(f)
        stopwords_path = "/content/vietnamese-stopwords.txt"
        # Tạo cặp dữ liệu huấn luyện - Cải thiện với multiple queries
        queries = [
            "Tình cảm anh em",
            "tấm gương hiếu thảo",
            "tình yêu thương ba mẹ",
            "lòng biết ơn cha mẹ"
        ]

        if os.path.exists(pairwise_data_file):
            print(f"1. Load dữ liệu pairwise từ {pairwise_data_file}...")

            # Khởi tạo sentence model trước
            sentence_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
            preparator = DataPreparator(sentence_model, stopwords_path)

            # Load dữ liệu
            pairwise_data, metadata = preparator.load_pairwise_data(pairwise_data_file)
            print(f"   ✓ Đã load {len(pairwise_data)} cặp từ file có sẵn")
            print(f"   ✓ Metadata: {metadata}")

        else:
            print("1. Tạo dữ liệu pairwise mới...")

            # Tạo dữ liệu pairwise
            pairwise_data = prepare_training_data(
                documents_path=documents_path,
                queries=queries,
                output_file=pairwise_data_file,
                stopwords_path=stopwords_path,
                relevance_method="cosine_ranking",
                max_pairs_per_query=1000
            )

            # Khởi tạo sentence model
            sentence_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2') #384 neurons
        # Tạo dataset và dataloader
        print("Đang chuẩn bị dataset...")
        dataset = RankNetDataset(pairwise_data, sentence_model)
        dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0)

        # Bước 2: Khởi tạo và huấn luyện mô hình RankNet
        print("Đang khởi tạo mô hình RankNet...")

        # Tính kích thước input (query embedding + document embedding)
        sample_query_emb = sentence_model.encode(["test"])[0]
        input_size = len(sample_query_emb) * 2  # query + document embeddings
        print(f"Input size: {input_size}")

        model = RankNet(input_size=input_size, hidden_size1=128, hidden_size2=64, dropout=0.5)
        optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

        print("Bắt đầu huấn luyện mô hình...")
        model = train_ranknet(model, dataloader, optimizer, device, num_epochs=20)

        # Lưu mô hình
        torch.save(model.state_dict(), 'ranknet_model.pth')
        print("Đã lưu mô hình vào ranknet_model.pth")

                # Bước 3: Đánh giá độ chính xác của mô hình
        print("\n" + "="*50)
        print("ĐÁNH GIÁ MÔ HÌNH")
        print("="*50)

        # Test queries
        test_queries = [
            "tình cảm anh em",
            "tình yêu thương ba mẹ",
        ]

        print("Đang đánh giá ranking...")
        ranking_results = evaluate_ranking(model, documents, test_queries, sentence_model, preparator, device, top_k=10)


        # Hiển thị kết quả top 10 cho mỗi query
        for query, result in ranking_results.items():
            print(f"\nQuery: '{query}'")
            print("Top 10 documents:")
            for i, (doc_id, doc_text, score) in enumerate(result['top_documents'], 1):
                print(f"{i:2d}. [ID: {doc_id}] Score: {score:.4f}")
                print(f"    Text: {doc_text[:100]}...")
                print()

        # Tính Pairwise Accuracy trên tập test
        print("Đang tính Pairwise Accuracy...")

        # Tạo test data từ test queries
        test_pairwise_data = prepare_training_data(
                documents_path=documents_path,
                queries=queries,
                output_file=pairwise_data_file,
                stopwords_path=stopwords_path,
                relevance_method="cosine_ranking",
                max_pairs_per_query=1000
            )  # Giới hạn để tính nhanh
        accuracy = calculate_pairwise_accuracy(model, test_pairwise_data, sentence_model, device)

        print(f"\nPairwise Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

        print("\n" + "="*50)
        print("HOÀN THÀNH!")
        print("="*50)

    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()



Sử dụng device: cuda
1. Load dữ liệu pairwise từ /content/document_pairs.json...
Đã load 1942 stopwords từ /content/vietnamese-stopwords.txt
Đã load 2664 cặp dữ liệu từ /content/document_pairs.json
   ✓ Đã load 2664 cặp từ file có sẵn
   ✓ Metadata: {'total_pairs': 2664, 'created_by': 'DataPreparator', 'format_version': '1.0', 'documents_path': '/content/cadao_tucngu_50_1.json', 'num_queries': 4, 'queries': ['Tình cảm anh em', 'tấm gương hiếu thảo', 'tình yêu thương ba mẹ', 'lòng biết ơn cha mẹ'], 'relevance_method': 'cosine_ranking', 'sentence_model': 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', 'max_pairs_per_query': 1000}
Đang chuẩn bị dataset...
Đang khởi tạo mô hình RankNet...
Input size: 768
Bắt đầu huấn luyện mô hình...


Epoch 1/20: 100%|██████████| 84/84 [01:23<00:00,  1.01it/s, Loss=0.6004, LR=0.001000]


Epoch 1/20, Average Loss: 0.6654
New best model saved with loss: 0.6654


Epoch 2/20: 100%|██████████| 84/84 [01:23<00:00,  1.01it/s, Loss=0.5259, LR=0.001000]


Epoch 2/20, Average Loss: 0.5929
New best model saved with loss: 0.5929


...

Epoch 7/20, Average Loss: 0.4653
New best model saved with loss: 0.4653


Epoch 8/20: 100%|██████████| 84/84 [01:22<00:00,  1.02it/s, Loss=0.6278, LR=0.001000]


Epoch 8/20, Average Loss: 0.4565
New best model saved with loss: 0.4565


Epoch 9/20:  31%|███       | 26/84 [00:26<01:00,  1.04s/it, Loss=0.3936, LR=0.001000]


KeyboardInterrupt: 

Tiến hành kiểm thử mô hình [RankNet] đã được huấn luyện, chúng ta sẽ kiểm tra đơn giản bằng cách - đầu vào chúng ta sẽ cho 2 tài liệu ( di ) và ( dj ) (ở dạng 2 vectors)

Sau đó chúng ta sẽ dùng mô hình [RankNet] để dự đoán thử xác suất tài liệu ( di ) liên quan với truy vấn ( q ) nhiều hơn ( dj )

In [41]:
"""Tiến hành kiểm thử mô hình [RankNet] đã được huấn luyện"""



if __name__ == "__main__":
    try:
        # Thiết lập device
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Sử dụng device: {device}")

        # Bước 1: Tải tài liệu và khởi tạo sentence-transformer
        documents_path = "/content/cadao_tucngu_50_1.json"
        pairwise_data_file = "/content/document_pairs.json"
        vector_pairs_file = "/content/vector_pairs.json"
        model_path = "/content/ranknet_model.pth"  # Đường dẫn mô hình đã huấn luyện

        # Load documents
        with open(documents_path, 'r', encoding='utf-8') as f:
            documents = json.load(f)
        stopwords_path = "/content/vietnamese-stopwords.txt"

        # Khởi tạo sentence model
        sentence_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
        preparator = DataPreparator(sentence_model, stopwords_path)

        # Load dữ liệu pairwise nếu có
        if os.path.exists(pairwise_data_file):
            print(f"1. Load dữ liệu pairwise từ {pairwise_data_file}...")
            pairwise_data, metadata = preparator.load_pairwise_data(pairwise_data_file)
            print(f"   ✓ Đã load {len(pairwise_data)} cặp từ file có sẵn")
        else:
            print("   ✗ Không tìm thấy file pairwise data")
            raise FileNotFoundError(f"File {pairwise_data_file} không tồn tại")

        # Bước 2: Tạo và lưu vector pairs vào file
        print("2. Tạo vector pairs và lưu vào file...")

        if len(pairwise_data) > 0:
            sample_pair = pairwise_data[0]  # Lấy cặp đầu tiên

            # Tạo embeddings cho query và documents
            query_embedding = sentence_model.encode([sample_pair['query']])[0]
            doc1_embedding = sentence_model.encode([sample_pair['doc1_text']])[0]  # Sửa key
            doc2_embedding = sentence_model.encode([sample_pair['doc2_text']])[0]  # Sửa key

            # Tạo combined vectors (query + document)
            di = np.concatenate([query_embedding, doc1_embedding])
            dj = np.concatenate([query_embedding, doc2_embedding])

            # Tạo dictionary để lưu
            vector_data = {
                "query": sample_pair['query'],
                "doc1_text": sample_pair['doc1_text'],
                "doc2_text": sample_pair['doc2_text'],
                "doc1_relevance": sample_pair['doc1_relevance'],
                "doc2_relevance": sample_pair['doc2_relevance'],
                "label": sample_pair['label'],
                "vector_di": di.tolist(),
                "vector_dj": dj.tolist(),
                "vector_size": len(di),
                "query_embedding_size": len(query_embedding),
                "doc_embedding_size": len(doc1_embedding)
            }

            # Lưu vào file JSON
            with open(vector_pairs_file, 'w', encoding='utf-8') as f:
                json.dump(vector_data, f, ensure_ascii=False, indent=2)

            print(f"   ✓ Đã lưu vector pairs vào {vector_pairs_file}")
            print(f"   ✓ Query: '{sample_pair['query']}'")
            print(f"   ✓ Document 1 (relevance: {sample_pair['doc1_relevance']}): '{sample_pair['doc1_text'][:100]}...'")
            print(f"   ✓ Document 2 (relevance: {sample_pair['doc2_relevance']}): '{sample_pair['doc2_text'][:100]}...'")
            print(f"   ✓ Label: {sample_pair['label']} (1.0: doc1 > doc2, 0.0: doc1 < doc2, 0.5: equal)")
            print(f"   ✓ Vector size: {len(di)}")

        else:
            print("   ✗ Không có dữ liệu pairwise để tạo vector")
            raise ValueError("Không có dữ liệu pairwise")

        # Bước 3: Đọc vector pairs từ file và kiểm tra
        print("\n3. Đọc vector pairs từ file và kiểm tra...")

        if os.path.exists(vector_pairs_file):
            with open(vector_pairs_file, 'r', encoding='utf-8') as f:
                loaded_data = json.load(f)

            # Chuyển đổi lại thành numpy arrays
            di_loaded = np.array(loaded_data['vector_di'])
            dj_loaded = np.array(loaded_data['vector_dj'])

            print(f"   ✓ Đã load vectors từ file")
            print(f"   ✓ Query: '{loaded_data['query']}'")
            print(f"   ✓ Document 1 relevance: {loaded_data['doc1_relevance']}")
            print(f"   ✓ Document 2 relevance: {loaded_data['doc2_relevance']}")
            print(f"   ✓ Ground truth label: {loaded_data['label']}")
            print(f"   ✓ Vector shape: {di_loaded.shape}")

            # Tính cosine similarity giữa hai vectors
            dot_product = np.dot(di_loaded, dj_loaded)
            norm_di = np.linalg.norm(di_loaded)
            norm_dj = np.linalg.norm(dj_loaded)
            cosine_sim = dot_product / (norm_di * norm_dj)
            print(f"   ✓ Cosine similarity giữa di và dj: {cosine_sim:.4f}")

        else:
            print(f"   ✗ Không tìm thấy file {vector_pairs_file}")
            raise FileNotFoundError(f"File {vector_pairs_file} không tồn tại")

        # Bước 4: Khởi tạo và load mô hình RankNet đã huấn luyện
        print("\n4. Khởi tạo và load mô hình RankNet...")

        # Tính kích thước input
        sample_query_emb = sentence_model.encode(["test"])[0]
        input_size = len(sample_query_emb) * 2
        print(f"   ✓ Input size: {input_size}")

        # Load weights và tự động detect architecture
        if os.path.exists(model_path):
            # Load checkpoint để kiểm tra kích thước
            checkpoint = torch.load(model_path, map_location=device)

            # Detect architecture từ checkpoint
            fc1_weight_shape = checkpoint['fc1.weight'].shape
            fc2_weight_shape = checkpoint['fc2.weight'].shape

            hidden_size1 = fc1_weight_shape[0]  # output của fc1
            hidden_size2 = fc2_weight_shape[0]  # output của fc2

            print(f"   ✓ Detected architecture: hidden_size1={hidden_size1}, hidden_size2={hidden_size2}")

            # Khởi tạo mô hình với đúng architecture
            model = RankNet(input_size=input_size, hidden_size1=hidden_size1, hidden_size2=hidden_size2, dropout=0.5)
            model.to(device)

            # Load weights
            model.load_state_dict(checkpoint)
            model.eval()
            print(f"   ✓ Đã load mô hình từ {model_path}")
        else:
            print(f"   ⚠ Không tìm thấy mô hình tại {model_path}, sử dụng mô hình mặc định")
            # Khởi tạo mô hình mặc định
            model = RankNet(input_size=input_size, hidden_size1=256, hidden_size2=128, dropout=0.2)
            model.to(device)
            print(f"   ✓ Khởi tạo mô hình mặc định với hidden_size1=256, hidden_size2=128")

        # Bước 5: Test mô hình với vectors
        print("\n5. Test mô hình RankNet...")

        with torch.no_grad():
            # Chuyển vectors thành tensor
            di_tensor = torch.tensor(di_loaded, dtype=torch.float32).unsqueeze(0).to(device)
            dj_tensor = torch.tensor(dj_loaded, dtype=torch.float32).unsqueeze(0).to(device)

            # Tính scores cho từng document
            score_i = model(di_tensor).item()
            score_j = model(dj_tensor).item()

            # Tính xác suất P(di > dj)
            diff = score_i - score_j
            probability_i_better_j = torch.sigmoid(torch.tensor(diff)).item()

            print(f"   ✓ Score document i: {score_i:.4f}")
            print(f"   ✓ Score document j: {score_j:.4f}")
            print(f"   ✓ Score difference (i - j): {diff:.4f}")
            print(f"   ✓ P(document i > document j): {probability_i_better_j:.4f} ({probability_i_better_j*100:.1f}%)")

            # So sánh với ground truth
            ground_truth_label = loaded_data['label']

            if ground_truth_label == 1.0:
                expected = "Document i should be ranked higher than document j"
                correct = probability_i_better_j > 0.5
            elif ground_truth_label == 0.0:
                expected = "Document j should be ranked higher than document i"
                correct = probability_i_better_j < 0.5
            else:  # label == 0.5
                expected = "Documents i and j should have similar ranks"
                correct = abs(probability_i_better_j - 0.5) < 0.1

            print(f"\n   📊 ĐÁNH GIÁ KẾT QUẢ:")
            print(f"   ✓ Ground truth: {expected}")
            print(f"   ✓ Model prediction: Document i có {probability_i_better_j*100:.1f}% khả năng rank cao hơn document j")
            print(f"   ✓ Prediction {'ĐÚNG' if correct else 'SAI'}: {'✓' if correct else '✗'}")

            # Thêm phân tích chi tiết
            print(f"\n   📈 PHÂN TÍCH CHI TIẾT:")
            print(f"   ✓ Document 1 relevance: {loaded_data['doc1_relevance']} -> Score: {score_i:.4f}")
            print(f"   ✓ Document 2 relevance: {loaded_data['doc2_relevance']} -> Score: {score_j:.4f}")

            if loaded_data['doc1_relevance'] > loaded_data['doc2_relevance']:
                relevance_says = "Document 1 should rank higher"
            elif loaded_data['doc1_relevance'] < loaded_data['doc2_relevance']:
                relevance_says = "Document 2 should rank higher"
            else:
                relevance_says = "Documents should rank equally"

            model_says = "Document 1 ranks higher" if score_i > score_j else "Document 2 ranks higher" if score_j > score_i else "Documents rank equally"

            print(f"   ✓ Based on relevance: {relevance_says}")
            print(f"   ✓ Based on model: {model_says}")

        # Bước 6: Test với nhiều cặp khác (optional)
        print("\n6. Test với nhiều cặp khác...")

        num_test_pairs = min(5, len(pairwise_data))
        correct_predictions = 0

        for i in range(num_test_pairs):
            test_pair = pairwise_data[i]

            # Tạo embeddings
            query_emb = sentence_model.encode([test_pair['query']])[0]
            doc1_emb = sentence_model.encode([test_pair['doc1_text']])[0]
            doc2_emb = sentence_model.encode([test_pair['doc2_text']])[0]

            # Tạo features
            feature1 = torch.tensor(np.concatenate([query_emb, doc1_emb]),
                                   dtype=torch.float32).unsqueeze(0).to(device)
            feature2 = torch.tensor(np.concatenate([query_emb, doc2_emb]),
                                   dtype=torch.float32).unsqueeze(0).to(device)

            with torch.no_grad():
                score1 = model(feature1).item()
                score2 = model(feature2).item()
                prob = torch.sigmoid(torch.tensor(score1 - score2)).item()

                # Kiểm tra prediction
                label = test_pair['label']
                if label == 1.0 and prob > 0.5:
                    correct_predictions += 1
                elif label == 0.0 and prob < 0.5:
                    correct_predictions += 1
                elif label == 0.5 and abs(prob - 0.5) < 0.1:
                    correct_predictions += 1

                print(f"   Pair {i+1}: P(doc1 > doc2) = {prob:.3f}, Label = {label}, {'✓' if (label == 1.0 and prob > 0.5) or (label == 0.0 and prob < 0.5) or (label == 0.5 and abs(prob - 0.5) < 0.1) else '✗'}")

        accuracy = correct_predictions / num_test_pairs
        print(f"\n   📊 Accuracy trên {num_test_pairs} cặp test: {accuracy:.2f} ({accuracy*100:.1f}%)")

        print("\n" + "="*60)
        print("HOÀN THÀNH KIỂM THỬ MÔ HÌNH RANKNET!")
        print("="*60)

    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()

Sử dụng device: cuda
Đã load 1942 stopwords từ /content/vietnamese-stopwords.txt
1. Load dữ liệu pairwise từ /content/document_pairs.json...
Đã load 2664 cặp dữ liệu từ /content/document_pairs.json
   ✓ Đã load 2664 cặp từ file có sẵn
2. Tạo vector pairs và lưu vào file...
   ✓ Đã lưu vector pairs vào /content/vector_pairs.json
   ✓ Query: 'Tình cảm anh em'
   ✓ Document 1 (relevance: 0): 'Nhiễu điều phủ lấy giá gương Chạy xe nhường nhịn, là thương chính mình...'
   ✓ Document 2 (relevance: 0): 'Anh ngồi tựa mạn thuyền rồng Thấy em cuốc cỏ trên đồng anh thương....'
   ✓ Label: 0.5 (1.0: doc1 > doc2, 0.0: doc1 < doc2, 0.5: equal)
   ✓ Vector size: 768

3. Đọc vector pairs từ file và kiểm tra...
   ✓ Đã load vectors từ file
   ✓ Query: 'Tình cảm anh em'
   ✓ Document 1 relevance: 0
   ✓ Document 2 relevance: 0
   ✓ Ground truth label: 0.5
   ✓ Vector shape: (768,)
   ✓ Cosine similarity giữa di và dj: 0.6428

4. Khởi tạo và load mô hình RankNet...
   ✓ Input size: 768
   ✓ Det